In [ ]:
# 带正则化的逻辑回归
# 2022年6月10日20:20:30：

In [ ]:
# 载入包
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

## 1.载入数据部分

In [ ]:
# 载入数据
path = 'ex2data2.txt'
data = pd.read_csv(path, header=None, names=['test 1', 'test 2', 'Accepted'])
data.head()

# add a ones column - this makes the matrix multiplication work out easier
# data.insert(0, 'Ones', 1)

# set X (training data) and y (target variable)
m = data.shape[0]
cols = data.shape[1]
X = data.iloc[:,0:cols-1]
y = data.iloc[:,cols-1:cols]


# convert to numpy arrays and initalize the parameter array theta
X = np.array(X.values)
y = np.array(y.values)
theta = np.zeros(28)

检查一下当前的变量状况

In [ ]:
print(theta)
X.shape, theta.shape, y.shape

In [ ]:
# 将原来只有两个feature的X转化成拥有28个feature的X
# 相当于octave实验中的mapFeature函数部分
def mapFeature(X):
    degree = 6
    new_X = np.ones((m, 1))
    for i in range(1, degree + 1):
        for j in range(0, i + 1):
            temp_X = pow(X[:, 0], (i-j)) * pow(X[:, 1], j)
            new_X = np.c_[new_X, temp_X]

    return new_X

new_X = mapFeature(X)
print('new_X.shape:\n', new_X.shape)
X = new_X
print(X)
# print(new_X)


## 2.下面计算损失和梯度

In [ ]:
# 定义sigmoid函数
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
# 测试一下sigmoid函数
# print(sigmoid(-4),sigmoid(0),sigmoid(4))

下面是代价函数：
$J\left( \theta  \right)=\frac{1}{m}\sum\limits_{i=1}^{m}{[-{{y}^{(i)}}\log \left( {{h}_{\theta }}\left( {{x}^{(i)}} \right) \right)-\left( 1-{{y}^{(i)}} \right)\log \left( 1-{{h}_{\theta }}\left( {{x}^{(i)}} \right) \right)]}$

定义损失计算函数

In [ ]:
def costReg(theta, X, y, learningRate):

    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)

    first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    second = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))
    reg = (learningRate / (2 * len(X))) * np.sum(np.power(theta[:,1:theta.shape[1]], 2))

    return np.sum(first - second) / len(X) + reg



定义计算梯度函数

In [ ]:
def gradientReg(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    
    parameters = int(theta.ravel().shape[1])
    grad = np.zeros(parameters)
    
    error = sigmoid(X * theta.T) - y
    
    for i in range(parameters):
        term = np.multiply(error, X[:,i])
        
        if (i == 0):
            grad[i] = np.sum(term) / len(X)
        else:
            grad[i] = (np.sum(term) / len(X)) + ((learningRate / len(X)) * theta[:,i])
    
    return grad

初始化一些参数

In [ ]:
learningRate = 1

测试一下求梯度

In [ ]:
# test_theta = [0,0,0]
# cost = costFunction(test_theta, X, y)
# grad = gradient(test_theta, X, y)
# print('Expected cost (approx): 0.693\nExpected gradients (approx):-0.1000 -12.0092 -11.2628')
# print('计算得到：', cost, grad)


print('Expected cost (approx): 0.693\n')
costReg(theta, X, y, learningRate)


gradientReg(theta, X, y, learningRate)


注意，我们实际上没有在这个函数中执行梯度下降，我们仅仅在计算一个梯度步长。在练习中，一个称为“fminunc”的Octave函数是用来优化函数来计算成本和梯度参数。由于我们使用Python，我们可以用SciPy的“optimize”命名空间来做同样的事情。

现在可以用SciPy's truncated newton（TNC）实现寻找最优参数。

In [ ]:
import scipy.optimize as opt
result2 = opt.fmin_tnc(func=costReg, x0=theta, fprime=gradientReg, args=(X, y, learningRate))
result2

最后，我们可以使用第1部分中的预测函数来查看我们的方案在训练数据上的准确度。

计算一下预测准确率

In [ ]:
def predict(theta, X):
    probability = sigmoid(X * theta.T)
    return [1 if x >= 0.5 else 0 for x in probability]

In [ ]:
theta_min = np.matrix(result2[0])
predictions = predict(theta_min, X)
correct = [1 if ((a == 1 and b == 1) or (a == 0 and b == 0)) else 0 for (a, b) in zip(predictions, y)]
accuracy = (sum(map(int, correct)) / len(correct))
print ('accuracy = {0}%'.format(accuracy))